# Use Case Description

Security teams often write detection rules or response playbooks (e.g., for SIEMs, SOAR platforms, or XDRs) but struggle to systematically map them to the right MITRE ATT&CK techniques. This limits the organization's visibility into detection coverage and hinders frameworks like D3FEND, ATT&CK Navigator, and threat-informed defense.

The goal is to automatically analyze each detection play's logic, objective, and context — and match it to the most relevant ATT&CK technique(s) with high confidence.

## Model used for this use case
Both Instruct Model and Reasoning Model can be used, but Instruct Model could work better as no complex reasoning is involved. In this example, we used the Instruct Model via SageMaker endpoint.

**Note**: Update the configuration variables below to match your deployment.

## Configuration
Update these variables to match your SageMaker deployment:

In [ ]:
# Update these variables to match your deployment
endpoint_name = 'foundation-sec-8b-endpoint'
aws_region = 'us-east-1'

print(f"Configuration:")
print(f"Endpoint: {endpoint_name}")
print(f"Region: {aws_region}")

## SetUp

The setup uses SageMaker endpoint instead of loading the model locally.

In [ ]:
import boto3
import json
import re
from IPython.display import display, Markdown

# Initialize SageMaker runtime client
sagemaker_runtime = boto3.client('sagemaker-runtime', region_name=aws_region)

print(f"Connected to SageMaker endpoint: {endpoint_name}")

In [ ]:
# Generation arguments for reproducible outputs
generation_args = {
    "max_new_tokens": 512,
    "temperature": None,  # None means deterministic (temperature=0)
    "repetition_penalty": 1.2,
    "do_sample": False,   # Deterministic sampling
    "use_cache": True,
}

print("Generation configuration:")
for key, value in generation_args.items():
    print(f"  {key}: {value}")

In [ ]:
def inference(prompt, system_prompt):
    """Inference function using SageMaker endpoint"""
    
    # Format the conversation for the model
    formatted_prompt = f"System: {system_prompt}\n\nUser: {prompt}\n\nAssistant: "
    
    # Prepare payload for SageMaker endpoint
    payload = {
        "inputs": formatted_prompt,
        "parameters": generation_args
    }
    
    try:
        response = sagemaker_runtime.invoke_endpoint(
            EndpointName=endpoint_name,
            ContentType='application/json',
            Body=json.dumps(payload)
        )
        
        result = json.loads(response['Body'].read().decode())
        
        # Handle different TGI response formats
        if isinstance(result, list) and len(result) > 0:
            generated_text = result[0].get('generated_text', '')
        elif isinstance(result, dict):
            generated_text = result.get('generated_text', str(result))
        else:
            generated_text = str(result)
        
        # Clean up the response (remove the original prompt if it's included)
        if generated_text.startswith(formatted_prompt):
            response_text = generated_text[len(formatted_prompt):].strip()
        else:
            response_text = generated_text.strip()
            
        # Remove any trailing special tokens
        response_text = re.sub(r'<\|.*?\|>$', '', response_text).strip()
        
        return response_text
        
    except Exception as e:
        print(f"Error invoking endpoint: {str(e)}")
        return f"Error: {str(e)}"

# Test the inference function
test_response = inference("Hello, can you help with MITRE ATT&CK mapping?", "You are a cybersecurity analyst.")
print("Test Response:")
print(test_response)

## MITRE Mapping Detection Plays

In [ ]:
SYSTEM_PROMPT = "You are a cybersecurity analyst provided with a series of threat detection plays. Each play includes its name and a short objective."

def make_prompt(detection_plays):
    return f'''Your task: For each detection play, map it to the most relevant MITRE ATT&CK technique. Include:
    1. MITRE technique ID and name
    2. Justification for the mapping
    
    ## DETECTION PLAYS
    {detection_plays}
    
    Respond in a structured format:
    - Play Name: <name>
    - MITRE Technique: <TXXXX: Technique Name>
    - Reason: <why it's a fit>'''

print("MITRE mapping functions ready!")

In [ ]:
detection_plays = """
- Name: Monitor Multiple Failed Login Attempts
  Objective: Detect brute-force attacks on user accounts by tracking repeated login failures.

- Name: Alert on PowerShell Script Execution
  Objective: Detect execution of potentially malicious PowerShell scripts by adversaries.

- Name: Dumping LSASS credentials
  Objective: Alert when attempts are made to dump credentials from LSASS process memory
"""

print("Sample detection plays loaded!")

In [ ]:
print("=== MITRE ATT&CK MAPPING ANALYSIS ===")
response = inference(make_prompt(detection_plays), SYSTEM_PROMPT)
display(Markdown(response))

## Advanced MITRE Mapping

Let's try with more complex detection plays:

In [ ]:
advanced_detection_plays = """
- Name: Detect Suspicious Registry Modifications
  Objective: Monitor for unauthorized changes to critical Windows registry keys that could indicate persistence mechanisms.

- Name: Network Lateral Movement Detection
  Objective: Identify unusual network connections between internal hosts that may indicate lateral movement.

- Name: Process Injection Detection
  Objective: Detect when one process injects code into another process, which is commonly used by malware.

- Name: Data Exfiltration via DNS
  Objective: Monitor for anomalous DNS queries that may be used to exfiltrate sensitive data.

- Name: Living off the Land Binary Usage
  Objective: Detect misuse of legitimate system binaries for malicious purposes.
"""

print("=== ADVANCED MITRE ATT&CK MAPPING ===")
advanced_response = inference(make_prompt(advanced_detection_plays), SYSTEM_PROMPT)
display(Markdown(advanced_response))

## Custom Detection Play Analysis

Add your own detection plays for analysis:

In [ ]:
# Add your own detection plays here
custom_detection_plays = """
# Add your detection plays in this format:
# - Name: Your Detection Play Name
#   Objective: Description of what this detection aims to identify
# 
# Example:
# - Name: Suspicious Cloud API Activity
#   Objective: Detect unusual patterns in cloud API calls that may indicate credential compromise
"""

if custom_detection_plays.strip() and not custom_detection_plays.startswith("# Add your"):
    print("=== ANALYZING YOUR CUSTOM DETECTION PLAYS ===")
    custom_response = inference(make_prompt(custom_detection_plays), SYSTEM_PROMPT)
    display(Markdown(custom_response))
else:
    print("Add your detection plays in the cell above to analyze them!")

## MITRE ATT&CK Coverage Analysis

Generate a coverage report for your detection plays:

In [ ]:
def generate_coverage_analysis(detection_plays):
    """Generate MITRE ATT&CK coverage analysis"""
    
    coverage_prompt = f"""Analyze the following detection plays and provide:
    
    1. A summary of MITRE ATT&CK techniques covered
    2. Identification of any coverage gaps in major attack phases
    3. Recommendations for additional detection plays to improve coverage
    4. Priority ranking of the current detections based on threat landscape
    
    Focus on the MITRE ATT&CK framework tactics: Initial Access, Execution, Persistence, 
    Privilege Escalation, Defense Evasion, Credential Access, Discovery, Lateral Movement, 
    Collection, Command and Control, and Exfiltration.
    
    DETECTION PLAYS:
    {detection_plays}"""
    
    coverage_system = "You are a threat intelligence analyst specializing in MITRE ATT&CK framework coverage assessment."
    
    return inference(coverage_prompt, coverage_system)

# Combine all detection plays for coverage analysis
all_plays = detection_plays + "\n" + advanced_detection_plays

print("=== MITRE ATT&CK COVERAGE ANALYSIS ===")
coverage_analysis = generate_coverage_analysis(all_plays)
display(Markdown(coverage_analysis))

## Bulk Detection Play Processing

Process multiple detection plays efficiently:

In [ ]:
def process_detection_plays_bulk(plays_list):
    """Process multiple detection plays and return structured results"""
    
    results = []
    
    for i, play in enumerate(plays_list, 1):
        print(f"\n=== Processing Detection Play {i}/{len(plays_list)} ===")
        
        # Format single play for analysis
        single_play_prompt = f"""Analyze this single detection play and provide:
        1. Primary MITRE ATT&CK technique mapping
        2. Secondary techniques (if applicable)
        3. Confidence level (High/Medium/Low)
        4. Implementation difficulty (Easy/Medium/Hard)
        
        DETECTION PLAY:
        {play}"""
        
        response = inference(single_play_prompt, SYSTEM_PROMPT)
        results.append({
            "play": play,
            "analysis": response
        })
        
        print(f"✓ Completed analysis for play {i}")
    
    return results

# Example bulk processing
sample_plays = [
    "- Name: Suspicious File Downloads\n  Objective: Monitor for downloads of potentially malicious file types from the internet.",
    "- Name: Unusual User Account Creation\n  Objective: Detect creation of new user accounts outside of normal business hours or by unauthorized users.",
    "- Name: Memory Dump Detection\n  Objective: Identify attempts to dump process memory which may contain sensitive information."
]

print("=== BULK PROCESSING DETECTION PLAYS ===")
bulk_results = process_detection_plays_bulk(sample_plays)

# Display results
for i, result in enumerate(bulk_results, 1):
    print(f"\n### Result {i}")
    display(Markdown(result["analysis"]))
    print("-" * 60)